In [1]:
import os
import string

def get_filtered_data(rootDir):
    ''' Return filtered data from root folder as array of strings and 
    the corresponding paths to the respective files
    '''
    allDirs = []
    allFiles = []
    files_txt = []
    data = []
    dataFiltered = []
    filesFiltered = []
    
    translator = str.maketrans('', '', string.punctuation)

    for root, dirs, files in os.walk(rootDir, topdown=False):
        for name in dirs:
            allDirs.append(os.path.join(root, name))

    for dP in allDirs:
        onlyFiles = [f for f in os.listdir(dP) if os.path.isfile(os.path.join(dP, f))]
        for oF in onlyFiles: allFiles.append(os.path.join(dP, oF))

    for aF in allFiles:
        #with open(aF, 'r', encoding = "ISO-8859-1") as newFile: 
        with open(aF, 'r', encoding = "utf-8", errors='replace') as newFile: 
            data.append(newFile.read())
            
    for ind, fileData in enumerate(data):
        filt = [d == '\x0c' for d in fileData]
        if sum(filt) < len(filt):
            s = fileData.lower()
            s = s.translate(translator)
            dataFiltered.append(s)
            filesFiltered.append(allFiles[ind])
            
    return (dataFiltered, filesFiltered)


def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [2]:
import nltk
import string
import os

rootDir = '/home/flo/Hackdays_Mannheim_2019/github/classified documents_txt'
(data, fileDirs) = get_filtered_data(rootDir)



In [3]:


dataFiltered = []

for d in data:
    datFilt = ''
    for w in d.split(' '):
        if w not in stopWords: datFilt += w
    dataFiltered.append(datFilt)
        

[nltk_data] Downloading package stopwords to /home/flo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['the', 'to', '-', 'pr', 'der', 'is', 'of', 'die', 'in', 'and', 'und', '']


In [42]:
# Extract different token arrays for all files
tokens = []
for d in data: tokens.append(nltk.word_tokenize(d))

In [3]:
# Extract ONE tokenarray for ALL files
globalData = ''
for d in data: globalData += d
globalTokens = nltk.word_tokenize(globalData)

In [32]:
# Remove stop words from data
from nltk.corpus import stopwords
nltk.download('stopwords')
#stopWords = set(stopwords.words('english'))
stopWords = ['the', 'to', '-', 'pr', 'der', 'is', 'of', 'die', 'in', 'and', 'und', '–', '•', '✔']
#print(stopWords)

globalTokens_filt = []

for gT in globalTokens:
    if gT not in stopWords: globalTokens_filt.append(gT)
    
    
for gT in globalTokens_filt:
    if ('ablass' in gT): print(gT)

[nltk_data] Downloading package stopwords to /home/flo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
ablassen
ablassen
ablassschrauben
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschrauben
ölablassschraube
ölablassschraube
ablassschrauben
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschrauben
ölablassschraube
ölablassschraube
ablassschrauben
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschrauben
ölablassschraube
ölablassschraube
ablassschrauben
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablassschraube
ölablasssc

In [28]:
from collections import Counter
count = Counter(globalTokens_filt)
print("len(count) = %s" % len(count))
print("most_common = %s" % count.most_common(40))
print("-------------------")

len(count) = 34233
most_common = [('1', 10993), ('2', 8988), ('operation', 8416), ('a', 7468), ('motor', 6969), ('0', 6602), ('frequency', 6296), ('setting', 6244), ('output', 5898), ('set', 5806), ('3', 5454), ('mit', 5367), ('when', 5334), ('4', 5330), ('inverter', 5182), ('parameter', 5098), ('for', 4966), ('7', 4463), ('be', 4294), ('or', 4236), ('signal', 4208), ('6', 4187), ('—', 4097), ('bei', 4050), ('terminal', 4036), ('sie', 4030), ('des', 3942), ('on', 3826), ('10', 3761), ('value', 3736), ('15', 3697), ('selection', 3580), ('x', 3468), ('für', 3442), ('function', 3432), ('5', 3430), ('mode', 3410), ('ist', 3232), ('time', 3196), ('8', 3166)]
-------------------


In [4]:
from collections import Counter

for i in range(len(data)):
    count = Counter(tokens[i])
    print("len(count) = %s" % len(count))
    print("most_common = %s" % count.most_common(20))
    print("-------------------")

len(count) = 6067
most_common = [('the', 8238), ('to', 3361), ('pr', 3202), ('is', 2445), ('of', 2144), ('operation', 2090), ('and', 1852), ('frequency', 1568), ('setting', 1560), ('✔', 1526), ('set', 1447), ('output', 1393), ('when', 1317), ('inverter', 1294), ('parameter', 1201), ('motor', 1199), ('a', 1099), ('in', 1066), ('signal', 1030), ('terminal', 998)]
-------------------
len(count) = 6392
most_common = [('–', 2459), ('der', 838), ('die', 640), ('und', 552), ('•', 452), ('2', 328), ('edr71', 315), ('mit', 297), ('in', 286), ('o', 279), ('bei', 276), ('1', 264), ('des', 263), ('225', 252), ('für', 239), ('betriebsanleitung', 222), ('drehstrommotoren', 216), ('explosionsgeschützte', 211), ('3', 194), ('werden', 187)]
-------------------
len(count) = 6392
most_common = [('–', 2459), ('der', 838), ('die', 640), ('und', 552), ('•', 452), ('2', 328), ('edr71', 315), ('mit', 297), ('in', 286), ('o', 279), ('bei', 276), ('1', 264), ('des', 263), ('225', 252), ('für', 239), ('betriebsa

most_common = [('die', 366), ('sie', 329), ('getriebe', 313), ('der', 263), ('–', 246), ('und', 231), ('1', 225), ('•', 194), ('mit', 172), ('spiroplan®', 153), ('betriebsanleitung', 151), ('w', 137), ('4', 137), ('typenreihen', 132), ('r7', 132), ('f7', 132), ('k7', 132), ('s7', 132), ('2', 129), ('in', 123)]
-------------------
len(count) = 4357
most_common = [('die', 366), ('sie', 329), ('getriebe', 313), ('der', 263), ('–', 246), ('und', 231), ('1', 225), ('•', 194), ('mit', 172), ('spiroplan®', 153), ('betriebsanleitung', 151), ('w', 137), ('4', 137), ('typenreihen', 132), ('r7', 132), ('f7', 132), ('k7', 132), ('s7', 132), ('2', 129), ('in', 123)]
-------------------
len(count) = 943
most_common = [('und', 75), ('der', 68), ('die', 51), ('zu', 36), ('sind', 34), ('oder', 32), ('rp02', 28), ('bei', 27), ('nm', 27), ('nmm2', 27), ('armatur', 24), ('pos', 24), ('ziva®z', 23), ('nicht', 23), ('gesa®', 22), ('ziva®g', 22), ('ist', 22), ('zesa®', 21), ('in', 20), ('betriebs', 19)]
----